In [ ]:
%load_ext autoreload
import os
import h5py as h5
import numpy as np

%autoreload 2
from zcu_tools import load_data  # noqa: E402
from zcu_tools.analysis.fluxdep import (  # noqa: E402
    InteractiveLines,
    InteractiveSelector,
    VisualizeSpet,
    calculate_energy,
    preprocess_data,
    search_in_database,
    fit_spectrum,
    dump_result,
    load_result,
)

In [ ]:
qub_name = "2DQ9/Q5"

cflx = None
eflx = None
period = None
s_spects = {}

os.makedirs(f"../result/{qub_name}/image", exist_ok=True)
os.makedirs(f"../result/{qub_name}/web", exist_ok=True)

In [ ]:
loadpath = f"../result/{qub_name}/params.json"
_, sp_params, cflx, period, allows = load_result(loadpath)

eflx = cflx + 0.5 * period

# Load Spectrum

In [ ]:
spect_path = "../Database/DT008/2025/03/Data_0318/DT008_TwoTone_flux_002.hdf5"
spectrum, fpts, mAs = load_data(spect_path, server_ip="100.105.52.95", port=4999)
mAs, fpts, spectrum = preprocess_data(mAs, fpts, spectrum)

s_spectrum = np.abs(spectrum - np.mean(spectrum, axis=0, keepdims=True))
s_spectrum /= np.std(s_spectrum, axis=0, keepdims=True)

In [ ]:
%matplotlib widget
actLine = InteractiveLines(s_spectrum, mAs, fpts, cflx, eflx)

In [ ]:
cflx, eflx = actLine.get_positions()
period = 2 * abs(eflx - cflx)
flxs = (mAs - cflx) / period + 0.5

cflx, eflx, period

In [ ]:
%matplotlib widget
actSel = InteractiveSelector(spectrum, flxs, fpts, threshold=5.0)

In [ ]:
s_flxs, s_fpts = actSel.get_positions()

In [ ]:
name = spect_path.split("/")[-1]
s_spects.update(
    {
        name: {
            "cflx": cflx,
            "spectrum": (flxs, fpts, spectrum),
            "points": (s_flxs, s_fpts),
        }
    }
)
s_spects.keys()

# Save & Load

In [ ]:
save_path = f"../result/{qub_name}/spect.hdf5"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
with h5.File(save_path, "x") as f:
    for path, val in s_spects.items():
        grp = f.create_group(path)
        grp.create_dataset("cflx", data=val["cflx"])
        spect_grp = grp.create_group("spectrum")
        spect_grp.create_dataset("flxs", data=val["spectrum"][0])
        spect_grp.create_dataset("fpts", data=val["spectrum"][1])
        spect_grp.create_dataset("data", data=val["spectrum"][2])
        points_grp = grp.create_group("points")
        points_grp.create_dataset("flxs", data=val["points"][0])
        points_grp.create_dataset("fpts", data=val["points"][1])

In [ ]:
load_path = f"../result/{qub_name}/spect.hdf5"
s_spects = {}
with h5.File(load_path, "r") as f:
    for key in f.keys():
        grp = f[key]
        cflx = grp["cflx"][()]
        spectrum = (
            grp["spectrum"]["flxs"][()],
            grp["spectrum"]["fpts"][()],
            grp["spectrum"]["data"][()],
        )
        points = (grp["points"]["flxs"][()], grp["points"]["fpts"][()])
        s_spects.update({key: {"cflx": cflx, "spectrum": spectrum, "points": points}})
s_spects.keys()

# Align cflx

In [ ]:
t_cflx = list(s_spects.values())[-1]["cflx"]
for spect in s_spects.values():
    shift = t_cflx - spect["cflx"]
    spect["cflx"] += shift
    spect["spectrum"] = (spect["spectrum"][0] + shift, *spect["spectrum"][1:])
    spect["points"] = (spect["points"][0] + shift, *spect["points"][1:])

In [ ]:
s_flxs = np.concatenate([s_spects[k]["points"][0] for k in s_spects.keys()])
s_fpts = np.concatenate([s_spects[k]["points"][1] for k in s_spects.keys()])
flx_bound = (
    np.nanmin([np.nanmin(s_spects[k]["spectrum"][0]) for k in s_spects.keys()]),
    np.nanmax([np.nanmax(s_spects[k]["spectrum"][0]) for k in s_spects.keys()]),
)
flxs = np.linspace(flx_bound[0], flx_bound[1], 500)
fpt_bound = (
    np.nanmin([np.nanmin(s_spects[k]["spectrum"][1]) for k in s_spects.keys()]),
    np.nanmax([np.nanmax(s_spects[k]["spectrum"][1]) for k in s_spects.keys()]),
)
fpts = np.linspace(fpt_bound[0], fpt_bound[1], 500)

# Fitting range

In [ ]:
# EJb = (1.0, 6.0)
# ECb = (0.8, 2.0)
# ELb = (0.01, 0.2)
EJb = (3.0, 15.0)
ECb = (0.2, 2.0)
ELb = (0.5, 2.0)

# Search in Database

In [ ]:
allows = {}
allows = {
    **allows,
    # "transitions": [(i, j) for i in (0, 1, 2) for j in range(8) if i < j],
    # "red side": [(i, j) for i in (0, 1, 2) for j in range(8) if i < j],
    # "blue side": [(i, j) for i in (0, 1, 2) for j in range(8) if i < j],
    # "mirror": [(i, j) for i in (0, 1, 2) for j in range(8) if i < j],
    # "transitions2": [(i, j) for i in (0, 1, 2) for j in range(8) if i < j],
    # "red side2": [(i, j) for i in (0, 1, 2) for j in range(8) if i < j],
    # "blue side2": [(i, j) for i in (0, 1, 2) for j in range(8) if i < j],
    # "mirror2": [(i, j) for i in (0, 1, 2) for j in range(8) if i < j],
}

In [ ]:
best_params, fig = search_in_database(
    s_flxs, s_fpts, "../Database/simulation/fluxonium_1.h5", allows, EJb, ECb, ELb
)
fig.savefig(f"../result/{qub_name}/image/search_result.png")

In [ ]:
energies = calculate_energy(flxs, *best_params, cutoff=40, evals_count=15)

In [ ]:
v_allows = {
    **allows,
    # "transitions": [(0, 3), (0, 2), (0, 4), (1, 2), (1, 3), (1, 5), (2, 5)],
    # "red side": [(2, 3), (1, 3)],
    # "transitions": [(i, j) for i in (0, 1, 2) for j in range(i + 1, 15)],
    # "transitions2": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
    # "blue side": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
    # "blue side2": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
    # "red side": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
    # "red side2": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
}

vs = VisualizeSpet(s_spects, s_flxs, s_fpts, flxs, energies, v_allows, auto_hide=True)
fig = vs.create_figure()
fig.update_layout(
    title=f"EJ/EC/EL = ({best_params[0]:.2f}, {best_params[1]:.2f}, {best_params[2]:.2f})",
    title_x=0.5,
)
fig.show()

# Scipy Optimization

In [ ]:
# fit the spectrumData
sp_params = fit_spectrum(s_flxs, s_fpts, best_params, allows, (EJb, ECb, ELb))

# print the results
print("Fitted params:", *sp_params)

In [ ]:
energies = calculate_energy(flxs, *sp_params, 40, 15)

In [ ]:
v_allows = {
    **allows,
    # "transitions": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
    # "transitions2": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
    # "blue side": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
    # "red side": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
}

vs = VisualizeSpet(s_spects, s_flxs, s_fpts, flxs, energies, v_allows, auto_hide=True)
fig = vs.create_figure()
fig.update_layout(
    title=f"EJ/EC/EL = ({sp_params[0]:.2f}, {sp_params[1]:.2f}, {sp_params[2]:.2f})",
    title_x=0.501,
)
fig.show()

# Save Parameters

In [ ]:
fig.write_html(f"../result/{qub_name}/web/spect_fit.html", include_plotlyjs="cdn")
fig.write_image(f"../result/{qub_name}/image/spect_fit.png", format="png")

In [ ]:
# dump the data
savepath = f"../result/{qub_name}/params.json"

dump_result(savepath, qub_name, sp_params, cflx, period, allows)